# The Sparks Foundation - GRIP
## Computer Vision & Internet of Things

# Task 3: Social Distancing detector

## Implement a real time Social Distancing detector which can identify the distance between two individuals in a crowd.

# All the concepts related to this project and the implementation part I learned from the [Pyimagesearch](https://www.pyimagesearch.com) Website.

### Created by: Jay Khade

In [ ]:
# importing libraries

import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
import imutils
import os

# structure of the Project

## Input Image --->  Object Detection --->  Pairwise distance between centroids ---> Comparision with safe distance ---> Results 

## definig constants

In [ ]:


# path to YOLO directory
MODEL_PATH = "yolo-coco"

# min probablity of filter to called as weak
MIN_CONF = 0.3

#non maxima supression threshold
NMS_THRESH =0.3

# min safe distance that  people can have between them in
MIN_DISTANCE = 50

# import yolo model
configPath = "yolo-coco/yolov3.cfg"
weightsPath = "yolo-coco/yolov3.weights"
labelsPath = "yolo-coco/coco.names"
videopath = "Social_Distancing.mp4"
output = "output.avi"
personIdx = 0
display = 0

## function to detect people

In [ ]:


def detect_people(frame, net, ln, personIdx=0):
  # grab the dimensions of the frame and  initialize the list of
  # results
  (H, W) = frame.shape[:2]
  results = []

  # construct a blob from the input frame and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes
  # and associated probabilities

  blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)


  # initialize our lists of detected bounding boxes, centroids, and
  # confidences, respectively
  boxes = []
  centroids = []
  confidences = []

  # loop over each of the layer outputs
  for output in layerOutputs:

    # loop over each of the detections
    for detection in output:

      # extract the class ID and confidence (i.e., probability)
      # of the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]

      # filter detections by (1) ensuring that the object
      # detected was a person and (2) that the minimum
      # confidence is met
      if classID == personIdx and confidence > MIN_CONF:

        # scale the bounding box coordinates back relative to
        # the size of the image, keeping in mind that YOLO
        # actually returns the center (x, y)-coordinates of
        # the bounding box followed by the boxes' width and
        # height
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")

        # use the center (x, y)-coordinates to derive the top
        # and left corner of the bounding box
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))

        # update our list of bounding box coordinates,
        # centroids, and confidences
        boxes.append([x, y, int(width), int(height)])
        centroids.append((centerX, centerY))
        confidences.append(float(confidence))

  # apply non-maxima suppression to suppress weak, overlapping
  # bounding boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)

  # ensure at least one detection exists
  if len(idxs) > 0:

    # loop over the indexes we are keeping
    for i in idxs.flatten():
      # extract the bounding box coordinates
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])
      # update our results list to consist of the person
      # prediction probability, bounding box coordinates,
      # and the centroid
      r = (confidences[i], (x, y, x + w, y + h), centroids[i])
      results.append(r)

  # return the list of results
  return results

In [ ]:
# loading labels
LABELS = open(labelsPath).read().strip().split("\n")

## YOLO Object detector

In [ ]:
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [ ]:
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
# initialize the video stream and pointer to output video file
print("[INFO] accessing video stream...")
vs = cv2.VideoCapture(videopath)
writer = None

## Frame Processing and Output

In [ ]:
while True:
  # Read the next frame from the file
  (grabbed, frame) = vs.read()
    
  # Grab frames from the video stream until we have reached the end frame
  if not grabbed:
    break
    
  # Resize the frame and then detect people (and only people) in it
  frame = imutils.resize(frame, width=700)
  results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

  # Initialize the set of indexes that violate the minimum social distance
  violate = set()

  # Ensure that there at least two people detections (required in order to
  # compute the pairwise distance maps)
  if len(results) >= 2:
    # Extract all centroids from the results and compute the Euclidean distance between all pairs of centroids
    centroids = np.array([r[2] for r in results])
    D = dist.cdist(centroids, centroids, metric="euclidean")
    
    # Loop over the upper triangular of the distance matrix
    for i in range(0, D.shape[0]):
      for j in range(i + 1, D.shape[1]):
      # Check to see if the distance between any two centroid pairs is less than the configured number
      # of pixels
        if D[i, j] < MIN_DISTANCE:
          # Update the violation set with the indexes of the centroid pairs
          violate.add(i)
          violate.add(j)
        
    # Loop over the results
    for (i, (prob, bbox, centroid)) in enumerate(results):
      # Extract the bounding box and centroid coordinates, then initialize the color of the annotation
      (startX, startY, endX, endY) = bbox
      (cX, cY) = centroid
      color = (0, 255, 0)
        
      # If the index pair exists within the violation set, then update the color
      if i in violate:
        color = (0, 0, 255)
        
      # Draw (1) a bounding box around the person and (2) the centroid coordinates of the person,
      cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
      cv2.circle(frame, (cX, cY), 5, color, 1)

    # Draw the total number of social distancing violations on the output frame
    text = "Social Distancing Violations: {}".format(len(violate))
    cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)
    
    # Check to see if the output frame should be displayed in the screen
    if display > 0:
      # Show the output frame
      cv2.imshow("Frame", frame)
      key = cv2.waitKey(1) & 0xFF
      # If the `q` key was pressed, break from the loop
      if key == ord("q"):
        break
        
    # If an output video file path has been supplied and the video writer has not been initialized, do so now
    if display != "" and writer is None:
      # Initialize the video writer
      fourcc = cv2.VideoWriter_fourcc(*"MJPG")
      writer = cv2.VideoWriter(output, fourcc, 25, (frame.shape[1], frame.shape[0]), True)
    # If the video writer is not None, write the frame to the output video file
    if writer is not None:
      writer.write(frame)